In [1]:
import pandas as pd

# Data Reading

## Read Turin Real Data
Real Turin Data is extracted from: [https://covid19.infn.it/iss/](https://covid19.infn.it/iss/)


In [40]:
turin_real_deaths = pd.read_csv('../data/turin-real/iss_bydate_torino_deceduti.csv', delimiter=',')
turin_real_positives = pd.read_csv('../data/turin-real/iss_bydate_torino_positivi.csv', delimiter=',')
turin_real_recoveries = pd.read_csv('../data/turin-real/iss_bydate_torino_ricoveri.csv', delimiter=',')
turin_real_icu = pd.read_csv('../data/turin-real/iss_bydate_torino_terapia_intensiva.csv', delimiter=',')

In [66]:
turin_real_recoveries

,date,casi,recoveries_7_days
0,2020-02-21,-9999,NaN
1,2020-02-22,-9999,NaN
2,2020-02-23,-9999,NaN
3,2020-02-24,-9999,1.142857
4,2020-02-25,-9999,1.142857
...,...,...,...
1225,2023-06-30,-9999,0.000000
1226,2023-07-01,-9999,0.142857
1227,2023-07-02,-9999,NaN
1228,2023-07-03,-9999,NaN


In [61]:
def join_based_on_min_date(df1, df2):
    min_date = min(df1['date'].min(), df2['date'].min())
    if min_date == df1['date'].min():
        left_df = df1
        right_df = df2
    else:
        left_df = df2
        right_df = df1

    merged = pd.merge(left_df, right_df, on='date', how='left')
    return merged.filter(regex=r'^(?!.*casi).*$', axis=1)

turin_real_deaths.columns = ['date','casi','deaths_7_days']
turin_real_positives.columns = ['date','casi','positives_7_days']
turin_real_recoveries.columns = ['date','casi','recoveries_7_days']
turin_real_icu.columns = ['date','casi','icu_7_days']

turin_real = join_based_on_min_date(turin_real_deaths,turin_real_positives)
turin_real = join_based_on_min_date(turin_real,turin_real_recoveries)
turin_real = join_based_on_min_date(turin_real,turin_real_icu)
turin_real

,date,positives_7_days,deaths_7_days,recoveries_7_days,icu_7_days
0,2020-02-20,NaN,NaN,NaN,NaN
1,2020-02-21,NaN,NaN,NaN,NaN
2,2020-02-22,NaN,NaN,NaN,NaN
3,2020-02-23,0.857143,NaN,NaN,NaN
4,2020-02-24,0.857143,NaN,1.142857,NaN
...,...,...,...,...,...
1231,2023-07-05,0.428571,NaN,NaN,NaN
1232,2023-07-06,0.428571,NaN,NaN,NaN
1233,2023-07-07,NaN,NaN,NaN,NaN
1234,2023-07-08,NaN,NaN,NaN,NaN
